In [ ]:
from utils import *

check_tensorflow_gpu()
K.clear_session()

proxy = SpatiotemporalCO2()

proxy.num_epochs = 200
proxy.batch_size = 12
proxy.verbose = 1

In [ ]:
proxy.load_data()
proxy.process_data(subsample=500)

In [ ]:
model0 = proxy.make_model()
model, fit = proxy.train(model0)

In [ ]:
y_train_pred = model.predict(proxy.X_train)
y_test_pred  = model.predict(proxy.X_test)
print('Train pred: {} | Test pred: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
k = 78
c = 1

fig, axs = plt.subplots(2, 12, figsize=(20,4))
for j in range(12):
    axs[0,j].imshow(proxy.y_train[k,j*5,:,:,c], 'jet')
    axs[1,j].imshow(y_train_pred[k,j*5,:,:,c], 'jet')
    axs[0,j].set(title='t={}'.format(j*5))
    for i in range(2):
        axs[i,j].set(xticks=[], yticks=[])
axs[0,0].set(ylabel='True'); axs[1,0].set(ylabel='Pred')
plt.show()

***
# END

In [25]:
np.argwhere(proxy.X_train[0,:,:,-1]!=0)

array([[26, 18],
       [47, 23]], dtype=int64)